In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip '../input/sberbank-russian-housing-market/train.csv.zip'
!unzip '../input/sberbank-russian-housing-market/test.csv.zip'
!unzip '../input/sberbank-russian-housing-market/sample_submission.csv.zip'

In [ ]:
train_df = pd.read_csv("./train.csv", parse_dates=['timestamp'])
dtype_df = train_df.dtypes.reset_index()
dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["qtd_atributos", "tipo"]
dtype_df.groupby("tipo").aggregate('count').reset_index()

* Transformar as colunas categóricas para rodar o XGBoost e ver as colunas mais relevantes.
* Usar ordinal encoding porque já tem muitas colunas e o objetivo é apenas filtrar algumas delas

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

cat = [a for a in train_df.columns if train_df[a].dtype == 'object']
train_df[cat] = encoder.fit_transform(train_df[cat])
dtype_df = train_df.dtypes.reset_index()
dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["qtd_atributos", "tipo"]
dtype_df.groupby("tipo").aggregate('count').reset_index()

* Verificar a proporção de dados faltantes em cada coluna

In [ ]:
missing = train_df.isnull().sum() * 100 / len(train_df)
missing_df = pd.DataFrame({"col": train_df.columns, "missing_values": missing})
missing_df = missing_df[missing_df.missing_values != 0]
missing_df.sort_values('missing_values', inplace=True, ascending=False)
missing_df.head(10)

In [ ]:
missing_df = missing_df[missing_df.missing_values > 40]
cols_to_remove = missing_df.col.to_list()
train_df.drop(cols_to_remove, inplace=True, axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
#removendo coluna tipo datetime
train_df.drop('timestamp', inplace=True, axis=1)

imputer = SimpleImputer(strategy='most_frequent')
itrain_df = pd.DataFrame(imputer.fit_transform(train_df))
itrain_df.columns = train_df.columns

In [ ]:
from xgboost import XGBRegressor

X = itrain_df.drop('price_doc', axis=1)
y = itrain_df['price_doc']

model = XGBRegressor(n_estimators=200, max_depth=13, random_state=987, eta=0.01)
model.fit(X, y)

feature_importance_df = pd.DataFrame({'col': X.columns, 'importance': model.feature_importances_})
feature_importance_df.sort_values('importance', inplace=True, ascending=False)
feature_importance_df.head(10)

In [ ]:
cols = feature_importance_df[feature_importance_df.importance > 0.016]
train = itrain_df[cols.col.to_list()]
train['price_doc'] = itrain_df.price_doc.values
train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

fig = plt.figure(figsize=(10,8))
corr = train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(230, 30, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

* Alta correlação entre os atributos que dizem respeito à área em que o imóvel está localizado, com exceção de 'ttk_km'.
* Avaliar atributos a serem removidos com base nas correlações entre eles e com 'price_doc'.

In [ ]:
cols_to_remove = ['cafe_count_3000_price_2500',
 'cafe_count_5000_price_2500',
 'cafe_count_2000',
 'office_sqm_5000',
 'cafe_count_1500_price_high']

train2 = train.drop(cols_to_remove, axis=1)

In [ ]:
fig = plt.figure(figsize=(6,6))
corr = train2.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(230, 30, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

In [ ]:
cols = ['id', 'full_sq', 'culture_objects_top_25', 'female_f', 'build_count_monolith', 'cafe_count_3000', 'sport_count_3000', 'price_doc']
df_train = pd.read_csv("./train.csv", usecols=cols)
cols.remove('price_doc')
df_test = pd.read_csv("./test.csv", usecols=cols)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train['build_count_monolith'].value_counts()

In [ ]:
df_train['build_count_monolith'].replace(np.NaN, 1.0, inplace=True)
df_test['build_count_monolith'].replace(np.NaN, 1.0, inplace=True)

In [ ]:
df_train['culture_objects_top_25'].unique()

In [ ]:
df_train['bin_culture_objects_top_25'] = np.where(df_train['culture_objects_top_25'] == 'yes', 1, 0)
df_train.drop('culture_objects_top_25', axis=1, inplace=True)
df_test['bin_culture_objects_top_25'] = np.where(df_test['culture_objects_top_25'] == 'yes', 1, 0)
df_test.drop('culture_objects_top_25', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_log_error

X = df_train.drop(['id','price_doc'], axis=1)
y = np.log(df_train.price_doc)

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=9)

lasso = Lasso()
lasso.fit(x_train, y_train)
y_pred = lasso.predict(x_test)
print('RMSLE: ', mean_squared_log_error(y_test,y_pred,squared=False))

In [ ]:
X = df_train.drop(['id','price_doc'], axis=1)
y = np.log(df_train.price_doc)

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=9)
xgb = XGBRegressor(n_estimators=286, max_depth=9, random_state=9, eta=.03)
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
print('RMSLE: ', mean_squared_log_error(y_test,y_pred,squared=False))

In [ ]:
test_ids = df_test.id.values
test_data = df_test.drop('id', axis=1)
predictions = np.exp(xgb.predict(test_data))
sub_preview = pd.DataFrame({'id': test_ids, 'price_doc': predictions})
sub_preview.head()

In [ ]:
submission = pd.read_csv("./sample_submission.csv")
submission['price_doc'] = predictions
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.head()